## ME447 Project 1
#### Amanda Schultz (amanda36) and Avanthi Obadage (obadage2)

### 2. 1/0 Knapsack Probelm 
#### Genetic Algorithm version

In [1]:
#initialize 
import numpy as np
from math import comb
A_data = np.load("A.npz") #dict format: n_items, capacity, item_values, item_weights
B_data = np.load("B.npz")

In [20]:
#Given
n_items = A_data["n_items"]
capacity = A_data["capacity"]
values = np.array(A_data["item_values"])
weights = np.array(A_data["item_weights"])


#### Representation 
##### Solutions are binary vectors; [1] is included and [0] is excluded

In [ ]:
# Representation
pop_size = 50 # how big is the population?
population = np.random.randint(2, size=(pop_size, n_items)) #some inital population with random objects

#Fitness
def fitness(individual):
    """
    A function that calculates the weight and value of an individual solution (parent) from the population
    """
    total_weight = np.sum(individual * weights)
    total_value = np.sum(individual * values)
    if total_weight > capacity:
        return 0  # overweight 
    return total_value

#Deterministic Variation Selection
num_parents = pop_size // 2 #next generation is half of previous
def select_parents(population, num_parents):
    """
    Selects the top solutions (parents) with highest fitness.
    """
    population = np.array(population)  # ensure it's an array
    fitness_vals = np.array([fitness(ind) for ind in population])
    top_idx = np.argsort(fitness_vals)[-num_parents:]  # largest fitness
    top_idx = top_idx[::-1]  
    return population[top_idx]

#Recombination
def crossover(parents, pop_size):
    """
    Crosses best parents to make offspring (new solutions) with (hopefully) better fitness
    """
    n_items = parents.shape[1]
    offspring = []
    while len(offspring) < pop_size:
        for i in range(len(parents)//2):
            p1 = parents[i]
            p2 = parents[-(i+1)]
            point = np.random.randint(1, n_items)
            c1 = np.concatenate([p1[:point], p2[point:]])
            c2 = np.concatenate([p2[:point], p1[point:]])
            offspring.append(c1)
            if len(offspring) < pop_size:
                offspring.append(c2)
    return np.array(offspring[:pop_size])

#Mutation
mutation_prob = 0.05 #some probability that a bit will flip 
def mutate(offspring, mutation_prob):
    """
    Randomly decides when a bit will flip to escape any local minima
    """
    mutation = np.random.rand(*offspring.shape) < mutation_prob
    return np.logical_xor(offspring, mutation).astype(int)

#Environmental Selection
def environmental_selection(offspring, pop_size):
    """
    Selects the top offspring individuals to form the next generation.
    """
    offspring = np.array(offspring) 
    fitness_vals = np.array([fitness(ind) for ind in offspring])
    top_idx = np.argsort(fitness_vals)[-pop_size:]  # largest fitness
    return offspring[top_idx]


# Run simulation!
generations = 50

for gen in range(generations):
    parents = select_parents(population, num_parents)
    
    # 4. Crossover
    offspring = crossover(parents, pop_size * 2)  # create extra to select from
    
    # 5. Mutation
    offspring = mutate(offspring, mutation_prob)
    
    # 6. Environmental selection (elitist)
    population = environmental_selection(offspring, pop_size)

# Solutions
fitness_vals = np.array([fitness(ind) for ind in population])
best_idx = np.argmax(fitness_vals)
best_solution = population[best_idx]
best_value = fitness_vals[best_idx]

print("Best solution:", best_solution)
print("Total value:", best_value)


Best solution: [0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0
 0 0 1 0 0 0 0 0 1 0 1 1 0]
Total value: 0


### 3. Minima of the Parabola
#### Genetic Algorithm version

In [96]:
#Initialization
import numpy as np
from math import comb
SEED = 42
rng = np.random.default_rng(seed=SEED)

In [ ]:
#Representation
pop_size = 100
num_parents = pop_size//2
num_offspring = comb(num_parents, 2)//2

def parabola_problem(individual):
    return 10*individual**2

In [ ]:
#Calculate Fitness
def fitness(pop):
    return [parabola_problem(element) for element in pop]

#Select Parents
def select_parents(pop, num_par):
    pop_fitness = fitness(pop)
    sorted_ind = np.argsort(pop_fitness)
    chosen_ind = sorted_ind[0:num_par]
    chosen_parents = pop[chosen_ind]
    return chosen_parents

#Crossover to generate offspring, taking the average between two randomly selected parents from the set
def crossover(parents, num_offspring):
    offspring = np.empty(num_offspring)
    for n in range(num_offspring):
        parent1_idx, parent2_idx = rng.choice(np.arange(num_parents), size = 2, replace = True)
        offspring[n] = (parents[parent1_idx] + parents[parent2_idx])/2
    return offspring

#Mutate offspring, using uniformly generated random numbers between -0.5 and 0.5
def mutate(offspring, prob):
    mutated_offspring = offspring.copy()

    mask = rng.uniform(size=offspring.shape)<prob
    drawn_numbers =  rng.uniform(-0.5, 0.5, size = offspring.shape)

    mutated_offspring[mask] = drawn_numbers[mask]

    return mutated_offspring

#Environmental selection, choose the fittest offspring to be the next generation
def environmental_selection(offspring):
    fitness_vals = fitness(offspring)
    sorted_ind = np.argsort(fitness_vals)
    chosen_ind = sorted_ind[0:pop_size]
    chosen_offspring = offspring[chosen_ind]
    return chosen_offspring

In [108]:
generations = 100
mutation_prob = 0.05
population = rng.uniform(-10,10, size = pop_size)

for gen in range(generations):
    parents = select_parents(population, num_parents)
    offspring = crossover(parents, num_offspring)
    #print(offspring)
    mutated_offspring = mutate(offspring, mutation_prob)
    #print(mutated_offspring)
    population = environmental_selection(mutated_offspring)

pop_fitness = fitness(population)
sorted_ind = np.argsort(pop_fitness)
print(population[sorted_ind[0]])

0.0


### 4. Minima of the Rotated Hyper-Ellipsoid
#### Genetic Algorithm Version

In [20]:
#Initialization
import numpy as np
from math import comb
SEED = 42
rng = np.random.default_rng(seed=SEED)

In [84]:
#Representation
pop_size = 50
num_parents = pop_size//2
num_offspring = comb(num_parents, 2)//2

def hyperellipsoid_problem(individual):
    f_x = (np.sqrt(3)/2 * (individual[0] - 3) + 0.5 * (individual[1] - 5))**2 + \
        5 * (np.sqrt(3)/2 * (individual[1] - 5) - 0.5 * (individual[0] - 3))**2
    return f_x

In [85]:
#Calculate Fitness
def fitness(pop):
    return [hyperellipsoid_problem(element) for element in pop]

#Select Parents
def select_parents(pop, num_par):
    pop_fitness = fitness(pop)
    sorted_ind = np.argsort(pop_fitness)
    chosen_ind = sorted_ind[0:num_par]
    chosen_parents = pop[chosen_ind]
    return chosen_parents

#Crossover to generate offspring, taking the average between two randomly selected parents from the set
def crossover(parents, num_offspring):
    offspring = np.empty((num_offspring,2))
    for n in range(num_offspring):
        parent1_idx, parent2_idx = rng.choice(np.arange(num_parents), size = 2, replace = True)
        parent1 = parents[parent1_idx]
        parent2 = parents[parent2_idx]
        offspring[n] = np.array((parent1[0],parent2[1]))
    return offspring

#Mutate offspring, by shifting the mutated offspring values by some delta within [-0.5,0.5) with some prob
def mutate(offspring, prob):
    mutated_offspring = offspring.copy()

    mask = rng.uniform(size=offspring.shape)<prob
    drawn_numbers =  rng.uniform(-0.5, 0.5, size = offspring.shape)
    mutated_offspring[mask] = offspring[mask]+drawn_numbers[mask]

    return mutated_offspring

#Environmental selection, choose the fittest offspring to be the next generation
def environmental_selection(offspring):
    fitness_vals = fitness(offspring)
    sorted_ind = np.argsort(fitness_vals)
    chosen_ind = sorted_ind[0:pop_size]
    chosen_offspring = offspring[chosen_ind]
    return chosen_offspring


In [86]:
generations = 100
mutation_prob = 0.05
population = rng.uniform(-10,10, size = (pop_size, 2))

for gen in range(generations):
    parents = select_parents(population, num_parents)
    offspring = crossover(parents, num_offspring)
    #print(offspring)
    mutated_offspring = mutate(offspring, mutation_prob)
    #print(mutated_offspring)
    population = environmental_selection(mutated_offspring)

pop_fitness = fitness(population)
sorted_ind = np.argsort(pop_fitness)
print(population[sorted_ind[0]], np.average(fitness(population)))

[2.99833485 4.99943137] 3.558821120355883e-06
